In [1]:
import cv2
import numpy as np
import time

In [6]:
def nothing(x):
    #any operation
    pass

#cap = cv2.VideoCapture("./input/input.mp4")
cap = cv2.VideoCapture(0)

# video settings
fps = int(cap.get(cv2.CAP_PROP_FPS))     
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))             
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')        
video = cv2.VideoWriter('./output/output.mp4', fourcc, fps, (w, h))  

#detection range 
xmin,xmax = 100,400 
ymin,ymax = 100,400 

#create trackbar
cv2.namedWindow("Trackbars",cv2.WINDOW_NORMAL)
cv2.createTrackbar("L-H", "Trackbars", 0, 180, nothing)
cv2.createTrackbar("L-S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L-V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U-H", "Trackbars", 180, 180, nothing)
cv2.createTrackbar("U-S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U-V", "Trackbars", 255, 255, nothing)

count = 0
f = 0
fps_time = 0

while True:
    _, frame = cap.read()
    
    #detection range   
    detframe = frame[ymin:ymax,xmin:xmax] 
    cv2.rectangle(frame,(xmin,ymin),(xmax,ymax),(0,255,255),2) 
    #convert hsv
    hsv = cv2.cvtColor(detframe, cv2.COLOR_BGR2HSV)
    
    l_h = cv2.getTrackbarPos("L-H", "Trackbars") 
    l_s = cv2.getTrackbarPos("L-S", "Trackbars") 
    l_v = cv2.getTrackbarPos("L-V", "Trackbars") 
    u_h = cv2.getTrackbarPos("U-H", "Trackbars") 
    u_s = cv2.getTrackbarPos("U-S", "Trackbars") 
    u_v = cv2.getTrackbarPos("U-V", "Trackbars") 
    
    #font setting
    font = cv2.FONT_HERSHEY_COMPLEX
    
    #prepare mask
    lower_red = np.array([l_h, l_s, l_v])
    upper_red = np.array([u_h, u_s, u_v])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel,iterations = 3)
    
    #contours detection
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    area_list = []
    for cnt in contours:
        #calculate Area 
        area = cv2.contourArea(cnt)
        area_list.append(area)
        #Contour approximation
        approx = cv2.approxPolyDP(cnt, 0.005*cv2.arcLength(cnt, True), True)
        #get landmark
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        
        #choose Aria range
        if area > 500:
            cv2.drawContours(detframe, [approx], 0, (255, 0, 255), 5)
            cv2.putText(detframe, 'area: {:.0f}'.format(area),(x-50, y), 
                        cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 0, 255), thickness=1)
    
    cv2.putText(frame, '%d, FPS : %.02f' % (count, 1.0 / (time.time() - fps_time)),
                            (5, 25), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 180, 0), 1)
    fps_time = time.time()
    #動画を1フレームずつ保存する  
    video.write(frame)                                               
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
    
    count += 1
    
cap.release()
video.release()
cv2.destroyAllWindows()